# ***Download neccessary package***

In [ ]:
!pip install transformers accelerate
!pip install underthesea
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 12.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***Data processing***

Dataset : https://www.kaggle.com/datasets/huhuyngun/vietnamese-chatbot-ver2

In [1]:
import pandas as pd
import torch , random , os
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
from underthesea import sent_tokenize,text_normalize,word_tokenize
from collections import Counter
from IPython.display import clear_output
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [2]:
# Đọc file data
df = pd.read_csv("/content/vi-QA.csv")
df.head(5)

,question,answers
0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,Có crush ai không,Có 1 bạn cùng lớp
2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,Có hay nói chuyện không,Hay nhắn tin messenger
4,Bạn kia có bắt chuyện trước không,Có đôi khi


In [3]:
# Kiểm tra dữ liệu rỗng
print(len(df))
print(df.isnull().sum())

110206
question      0
answers     276
dtype: int64


In [4]:
# Bỏ dữ liệu rỗng
df = df.dropna()
print(df.isnull().sum())
print(len(df))

question    0
answers     0
dtype: int64
109930


In [5]:
# Thực hiện tiền xử lý dữ liệu
# Chuyển dữ liệu pandas về list
ques = df['question'].values
ans = df['answers'].values

# Chuẩn hóa dữ liệu tiếng việt tiêu chuẩn
norm_ques = [text_normalize(sentence) for sentence in df['question'].values]
norm_ans = [text_normalize(sentence) for sentence in df['answers'].values]

# Sử dụng word segmentation
seg_ques = [word_tokenize(sentence, format="text") for sentence in norm_ques]
seg_ans = [word_tokenize(sentence, format="text") for sentence in norm_ans]

In [6]:
# Biểu diễn dữ liệu dưới dạng bag-of-word
# Tạo từ điển bằng counter()
tokenizer = Counter()
tokenizer.update([word.lower() for sentence in seg_ques + seg_ans
                  for word in sentence.split()])
word2index = {word: index + 4 for index, word in enumerate(tokenizer)} # Vocabulary
word2index['<pad>'] = 0 # dùng để padding những câu ngắn bằng với câu dài
word2index['<sos>'] = 1 # vị trí bắt đầu của một câu
word2index['<eos>'] = 2 # vị trí kết thúc của một câu
word2index['<unk>'] = 3 # các từ không nằm trong từ điển
index2word = {index: word for word, index in word2index.items()}

# Biểu diễn các câu bằng vị trí có trong từ điển
# question = []
# for sentence in seg_ques:
#   senten = []
#   for word in sentence.split():
#     if word in word2index:     # Chỉ thêm các từ có trong từ điển
#       senten.append(word2index[word])
#   question.append(senten)

# answer = []
# for sentence in seg_ans:
#   senten = []
#   for word in sentence.split(): 
#     if word in word2index:      # Chỉ thêm các từ có trong từ điển
#       senten.append(word2index[word]) 
#   answer.append(senten)

question = [[word2index[word.lower()] for word in sentence.split()]
                for sentence in seg_ques]
answer = [[word2index[word.lower()] for word in sentence.split()]
              for sentence in seg_ans]

# Thêm thẻ SOS và EOS câu trả lời tương ứng
for i in range(len(answer)):
    answer[i] = [word2index['<sos>']] + answer[i] + [word2index['<eos>']]

# Điều chỉnh độ dài của các câu hỏi và câu trả lời về cùng một độ dài
max_length = max(len(x) for x in answer + question) # Câu có độ dài lớn nhất trong bộ dữ liệu
pad_ques = [sequence[:max_length] + [word2index['<pad>']] * (max_length - len(sequence[:max_length])) 
                for sequence in question]
pad_ans = [sequence[:max_length] + [word2index['<pad>']] * (max_length - len(sequence[:max_length]))
              for sequence in answer]

In [7]:
print(pad_ques[0])
print(pad_ans[0])

[4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1276, 211, 6879, 730, 211, 3571, 2219, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
# Hàm chia dữ liệu thành ( train : 80% , validation : 10% , test : 10% )  
def train_val_test(data):
  train = data[:int(len(df)*0.8)]
  val = data[len(train):-int(len(df)*0.1)]
  test = data[len(train + val):]
  return train,val,test

X_train,X_val,X_test = train_val_test(pad_ques)
y_train,y_val,y_test = train_val_test(pad_ans)
len(X_train),len(X_val),len(X_test)

(87944, 10993, 10993)

In [9]:
print(X_train[0])
print(y_train[0])

[4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1276, 211, 6879, 730, 211, 3571, 2219, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# Tạo dataset và dataloader
class QADataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = torch.LongTensor(questions)
        self.answers = torch.LongTensor(answers)
    
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, index):
        return self.questions[index], self.answers[index]

train_dataset = QADataset(X_train, y_train)
val_dataset = QADataset(X_val, y_val)
test_dataset = QADataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=32 , shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32 , shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32 , shuffle=True)

In [11]:
# Kiểm thử dataloader
for i,(ques,ans) in enumerate(train_dataloader):
  print(f"i = {i} , ques = {ques.shape} , ans = {ans.shape}")
  break

i = 0 , ques = torch.Size([32, 83]) , ans = torch.Size([32, 83])


In [12]:
# Hàm đổi số về chữ
def decode_word(index2word,enc_word):
  dec_word = []
  for sent in enc_word:
    sentence = []
    for idx in sent:
      # list(sent).index(idx)
      if index2word[idx] == '<eos>' or index2word[idx] == '<sos>' or index2word[idx] == '<pad>':
        continue
      sentence.append(index2word[idx])
    dec_word.append(sentence)
  return dec_word

# ***Seq2Seq model***

In [25]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size,
                 hidden_size, num_layers , p):
      super(Encoder,self).__init__()
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.dropout = nn.Dropout(p)
      self.embedding = nn.Embedding(input_size,embedding_size)
      self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,dropout = p) 

    def forward(self, input):
      embedding =  self.dropout(self.embedding(input)) # shape = (max_len,batch_size,embedding_size)
      outputs , (hidden,cell) = self.rnn(embedding)
      return hidden,cell

class Decoder(nn.Module):
    def __init__(self, input_size, output_size, embedding_size,
                 hidden_size, num_layers , p):
      super(Decoder,self).__init__()
      self.input_size = input_size
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.dropout = nn.Dropout(p)
      self.embedding = nn.Embedding(input_size,embedding_size)
      self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers, dropout = p) 
      self.fc = nn.Linear(hidden_size,output_size)

    def forward(self,input , hidden , cell):
      input = input.unsqueeze(0) # shape = (1, batch_size)
      embedding = self.dropout(self.embedding(input)) # shape = (1,batch_size,embedding_size)
      outputs,(hidden,cell) = self.rnn(embedding,(hidden,cell)) # shape = (1,batch_size,hidden_size)
      predictions = self.fc(outputs) # shape = (1,batch_size,output_size)
      return predictions.squeeze(0),hidden,cell

class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder):
      super(Seq2Seq,self).__init__()
      self.encoder = encoder
      self.decoder = decoder 

    def forward(self,ques,ans,teacher = 0.5):
      batch_size = ques.shape[1]
      ans_len = ans.shape[0]
      voc_len = len(word2index)

      outputs = torch.zeros(ans_len,batch_size,voc_len).to(device)
      hidden,cell = self.encoder(ques)
      
      # Lấy token đầu tiên
      x = ans[0]
      for t in range(1,ans_len):
        output,hidden,cell = self.decoder(x,hidden,cell)
        outputs[t] = output
        best_guess = output.argmax(1)
        x = ans[t] if random.random() < teacher else best_guess
      return outputs 

In [26]:
# Training model
num_epochs = 3
lr = 0.001

# Model hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(word2index)
input_size_decoder = len(word2index)
ouput_size = len(word2index)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layer = 2
enc_dropout = 0.1
dec_dropout = 0.1

input_size = len(word2index)
output_size = len(word2index)
hidden_size = 256


In [27]:
encoder_net = Encoder(input_size_encoder,encoder_embedding_size,hidden_size
                      ,num_layer,enc_dropout)
decoder_net = Decoder(input_size_decoder,ouput_size,decoder_embedding_size,hidden_size
                      ,num_layer,dec_dropout)
seq2seq = Seq2Seq(encoder_net,decoder_net).to(device)
# coalb drive : /content/drive/MyDrive/NLP/checkpoint/
# local file : ./checkpoint/
#Lấy model checkpoint
if os.path.isfile('/content/drive/MyDrive/NLP/checkpoint/seq2seq_10.pth'):
    if device.type == 'cpu':
        seq2seq.load_state_dict(torch.load('/content/drive/MyDrive/NLP/checkpoint/seq2seq_10.pth',map_location=torch.device('cpu')))
    else:
        seq2seq.load_state_dict(torch.load('/content/drive/MyDrive/NLP/checkpoint/seq2seq_10.pth'))

optimizer = optim.Adam(seq2seq.parameters(),lr = lr)
criterion = nn.CrossEntropyLoss(ignore_index = word2index['<pad>'])

In [28]:
print(seq2seq)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding): Embedding(61587, 300)
    (rnn): LSTM(300, 256, num_layers=2, dropout=0.1)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding): Embedding(61587, 300)
    (rnn): LSTM(300, 256, num_layers=2, dropout=0.1)
    (fc): Linear(in_features=256, out_features=61587, bias=True)
  )
)


In [ ]:
# Training model
# coalb drive : /content/drive/MyDrive/NLP/checkpoint/
# local file : ./checkpoint/
seq2seq.train()
if not os.path.isfile('/content/drive/MyDrive/NLP/checkpoint/seq2seq_10.pth'):
  for epoch in range(3,10):
      print(f"Epoch [{epoch+1}/{10}]")
      for i,(ques,ans) in enumerate(train_dataloader):
          input_question = ques.permute(1,0).to(device)
          target = ans.permute(1,0).to(device)

          output = seq2seq(input_question,target) # (ans_len,batch_size,ouput_dim)
          output = output[1:].reshape(-1,output.shape[2])
          target = target[1:].reshape(-1)

          optimizer.zero_grad()
          loss = criterion(output,target)
          loss.backward()
          optimizer.step()

          if i % 100 == 0:
              print(f"Step [{i}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

      torch.save(seq2seq.state_dict(), f"/content/drive/MyDrive/NLP/checkpoint/seq2seq_{epoch+1}.pth")
      print(f'Saving seq2seq_{epoch+1}.pth')
    

In [ ]:
from torchtext.data.metrics import bleu_score
# Đánh giá mô hình bằng độ đo BLEU
evaluate = True
seq2seq.eval()
preds = []
refs = []
total = len(test_dataloader)
######
if evaluate:
  with torch.no_grad():
      for i, (ques, ans) in enumerate(test_dataloader):
          print("Evaluation state : %0.3f %% " % (i*100/total))
          input_seq = ques.to(device)
          target_seq = ans.to(device)

          output_seq = seq2seq(input_seq.permute(1,0), input_seq.permute(1,0), teacher=0) # no teacher forcing during evaluation
          output_seq = output_seq.permute(1,0,2).argmax(dim=-1).cpu().numpy()
          target_seq = target_seq.cpu().numpy()

          # output_seq = [[index2word[idx] for idx in sent] for sent in output_seq]
          target_seq = decode_word(index2word,target_seq)
          output_seq = decode_word(index2word,output_seq)

          preds.extend(output_seq)
          refs.extend(target_seq)
          clear_output(wait=True)
  print('Done.!')

Done.!


In [ ]:
# Kiểm tra câu dự đoán
if evaluate:
  print(refs[:10])
  print(preds[:10])

[['fredericia', '(', 'đan_mạch', ')', ',', 'berlin', ',', 'stockholm', ',', 'hamburg', ',', 'frankfurt', ',', 'helsinki', 'và', 'emden'], ['bằng', 'cách', 'kích_thích', 'sự', 'phát_triển', 'của', 'mạng_lưới', 'sản_xuất', 'hạt_giống', 'và', 'đại_lý', 'nông_nghiệp', 'để', 'phân_phối', 'và', 'tiếp_thị'], ['không_gian', 'thời_gian', 'ngắn', 'nhất', 'giữa', 'hai', 'thời_gian', 'không_gian', 'sự_kiện'], ['dick', 'clark'], ['bởi', 'một', 'khoản', 'phí', 'cho', 'mỗi', 'đơn_vị', 'thông_tin', 'được', 'truyền', ',', 'chẳng_hạn', 'như', 'ký', 'tự', ',', 'gói', 'hoặc', 'tin', 'nhắn'], ['điều_lệ'], ['singlet'], ['18', '%'], ['ch', 'quashev'], ['louis', 'agassiz'], ['sự', 'nhiệt_tình', 'của', 'giáo_viên'], ['westwood', 'one'], ['1'], ['charles', 'brenton_huggins', 'và', 'janet_rowley'], ['lá'], ['công_nhân', ',', 'nhà_tư_bản', '/', 'chủ', 'doanh_nghiệp', ',', 'chủ', 'nhà'], ['những', 'điều', 'là', 'vấn_đề', 'tùy', 'chỉnh', 'hoặc', 'kỳ_vọng'], ['nhân_loại'], ['1900'], ['sự', 'bắt_buộc'], ['điều', '102

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
# Compute the BLEU score
if evaluate:
  chencherry = SmoothingFunction()
  score = corpus_bleu(refs,preds,  smoothing_function=chencherry.method2)  
  print(f"BLEU: {score*100:.4f}")

BLEU: 0.0003


# ***Seq2Seq Attention model***

In [29]:
class EncoderAttention(nn.Module):
    def __init__(self, input_size, embedding_size,
                 hidden_size, num_layers , p):
      super(EncoderAttention,self).__init__()
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.dropout = nn.Dropout(p)
      self.embedding = nn.Embedding(input_size,embedding_size)
      self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers
                         ,bidirectional = True,dropout = p)
      self.fc_hidden = nn.Linear(hidden_size*2,hidden_size)
      self.fc_cell = nn.Linear(hidden_size*2,hidden_size)

    def forward(self, input):
      embedding =  self.dropout(self.embedding(input)) # shape = (max_len,batch_size,embedding_size)
      encoder_states , (hidden,cell) = self.rnn(embedding)
      hidden = self.fc_hidden(torch.cat((hidden[0:1],hidden[1:2]),dim = 2))
      cell = self.fc_cell(torch.cat((cell[0:1],cell[1:2]),dim = 2))

      return encoder_states,hidden,cell

class DecoderAttention(nn.Module):
    def __init__(self, input_size, output_size, embedding_size,
                 hidden_size, num_layers , p):
      super(DecoderAttention,self).__init__()
      self.input_size = input_size
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.dropout = nn.Dropout(p)
      self.embedding = nn.Embedding(input_size,embedding_size)
      self.rnn = nn.LSTM(hidden_size*2 + embedding_size,
                         hidden_size,num_layers, dropout = p) 
      self.energy = nn.Linear(hidden_size*3,1)
      self.sofmax = nn.Softmax(dim=0)
      self.relu = nn.ReLU()
      self.fc = nn.Linear(hidden_size,output_size)

    def forward(self,input , encoder_states , hidden , cell):
      input = input.unsqueeze(0) # shape = (1, batch_size)
      embedding = self.dropout(self.embedding(input)) # shape = (1,batch_size,embedding_size)
      seq_len = encoder_states.shape[0]
      h_reshaped = hidden.repeat(seq_len,1,1)
      energy = self.relu(self.energy(torch.cat((h_reshaped,encoder_states),dim = 2)))
      attention = self.sofmax(energy).permute(1,2,0)
      encoder_states = encoder_states.permute(1,0,2)
      context_vector = torch.bmm(attention,encoder_states).permute(1,0,2)
      rnn_input = torch.cat((context_vector,embedding),dim=2)
      outputs,(hidden,cell) = self.rnn(rnn_input,(hidden,cell)) # shape = (1,batch_size,hidden_size)
      predictions = self.fc(outputs) # shape = (1,batch_size,output_size)
      return predictions.squeeze(0),hidden,cell

class Seq2SeqAttention(nn.Module):
    def __init__(self,encoder,decoder):
      super(Seq2SeqAttention,self).__init__()
      self.encoder = encoder
      self.decoder = decoder 

    def forward(self,ques,ans,teacher = 0.8):
      batch_size = ques.shape[1]
      ans_len = ans.shape[0]
      voc_len = len(word2index)

      outputs = torch.zeros(ans_len,batch_size,voc_len).to(device)
      encoder_states,hidden,cell = self.encoder(ques)
      
      # Lấy token đầu tiên
      x = ans[0]
      for t in range(1,ans_len):
        output,hidden,cell = self.decoder(x,encoder_states,hidden,cell)
        outputs[t] = output
        best_guess = output.argmax(1)
        x = ans[t] if random.random() < teacher else best_guess
      return outputs 

In [30]:
# Training model
num_epochs = 3
lr = 0.001

# Model hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(word2index)
input_size_decoder = len(word2index)
ouput_size = len(word2index)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layer = 1
enc_dropout = 0
dec_dropout = 0

In [31]:
encoder_attention = EncoderAttention(input_size_encoder,encoder_embedding_size,hidden_size
                      ,num_layer,enc_dropout)
decoder_attention = DecoderAttention(input_size_decoder,ouput_size,decoder_embedding_size,hidden_size
                      ,num_layer,dec_dropout)
seq2seqAttention = Seq2SeqAttention(encoder_attention,decoder_attention).to(device)
# coalb drive : /content/drive/MyDrive/NLP/checkpoint/
# local file : ./checkpoint/
#Lấy model checkpoint
checkpoint_path = '/content/drive/MyDrive/NLP/checkpoint/seq2seqAttention_19.pth'
if os.path.isfile(checkpoint_path):
    if device.type == 'cpu':
        seq2seqAttention.load_state_dict(torch.load(checkpoint_path,map_location=torch.device('cpu')))
    else:
        seq2seqAttention.load_state_dict(torch.load(checkpoint_path))

optimizer = optim.Adam(seq2seqAttention.parameters(),lr = lr)
criterion = nn.CrossEntropyLoss(ignore_index = word2index['<pad>'])

In [32]:
print(seq2seqAttention)

Seq2SeqAttention(
  (encoder): EncoderAttention(
    (dropout): Dropout(p=0, inplace=False)
    (embedding): Embedding(61587, 300)
    (rnn): LSTM(300, 1024, bidirectional=True)
    (fc_hidden): Linear(in_features=2048, out_features=1024, bias=True)
    (fc_cell): Linear(in_features=2048, out_features=1024, bias=True)
  )
  (decoder): DecoderAttention(
    (dropout): Dropout(p=0, inplace=False)
    (embedding): Embedding(61587, 300)
    (rnn): LSTM(2348, 1024)
    (energy): Linear(in_features=3072, out_features=1, bias=True)
    (sofmax): Softmax(dim=0)
    (relu): ReLU()
    (fc): Linear(in_features=1024, out_features=61587, bias=True)
  )
)


In [ ]:
# Training model
if not os.path.isfile(checkpoint_path):
  seq2seqAttention.train()
  for epoch in range(3,10):
    print(f"Epoch [{epoch+1}/{10}]")
    for i,(ques,ans) in enumerate(train_dataloader):
      input_question = ques.permute(1,0).to(device)
      target = ans.permute(1,0).to(device)

      output = seq2seqAttention(input_question,target) # (ans_len,batch_size,ouput_dim)
      output = output[1:].reshape(-1,output.shape[2])
      target = target[1:].reshape(-1)

      optimizer.zero_grad()
      loss = criterion(output,target)
      loss.backward()
      optimizer.step()

      if i % 100 == 0:
          print(f"Step [{i}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

    torch.save(seq2seq.state_dict(), f"/content/drive/MyDrive/NLP/checkpoint/seq2seqAttention_{epoch+1}.pth")
    print(f'Saving seq2seqAttention_{epoch+1}.pth')

In [ ]:
from torchtext.data.metrics import bleu_score
# Đánh giá mô hình bằng độ đo BLEU
evaluate = True
preds = []
refs = []
total = len(test_dataloader)
######
if evaluate:
  with torch.no_grad():
      for i, (ques, ans) in enumerate(test_dataloader):
          print("Evaluation state : %0.3f %% " % (i*100/total))
          input_seq = ques.to(device)
          target_seq = ans.to(device)

          output_seq = seq2seqAttention(input_seq.permute(1,0), input_seq.permute(1,0), teacher=0) # no teacher forcing during evaluation
          output_seq = output_seq.permute(1,0,2).argmax(dim=-1).cpu().numpy()
          target_seq = target_seq.cpu().numpy()

          # output_seq = [[index2word[idx] for idx in sent] for sent in output_seq]
          target_seq = decode_word(index2word,target_seq)
          output_seq = decode_word(index2word,output_seq)

          preds.extend(output_seq)
          refs.extend(target_seq)
          clear_output(wait=True)

  print('Done.!')

Done.!


In [ ]:
# Kiểm tra câu dự đoán
if evaluate:
  print(refs[:10])
  print(preds[:10])

[['pep'], ['chủ_nghĩa', 'hiện_thực', 'xã_hội_chủ_nghĩa'], ['vàng'], ['sage', 'gateshead_music_and_arts_centre'], ['miller-boyett', 'productions'], ['4'], ['3'], ['scandinavia'], ['khí_quyển', 'oxy'], ['bjorn', 'waldegård', ',', 'hannu_mikkola', ',', 'tommi_mäkinen', ',', 'shekhar_mehta', ',', 'carlos_sainz', 'và', 'colin_mcrae']]
[['người', 'gièm_pha'], ['của', 'serres'], ['sự', 'phát_triển', 'của', 'con_người'], ['pháp'], ['người', 'phẳng'], [], [], ['để', 'vinh_danh', 'trong', 'nước_lớn'], ['các', 'cuộc', 'tấn_công', 'tàu_ngầm'], ['họ', 'đã']]


In [ ]:
# Compute the BLEU score
if evaluate:
  chencherry = SmoothingFunction()
  score = corpus_bleu(refs,preds, smoothing_function=chencherry.method2)
  print(f"BLEU score: {score*100:.4f}")

BLEU score: 0.0089


# ***Bert model***

In [18]:
from datasets import Dataset
# Split the data into train, validation, and test sets
train_size = int(0.8 * len(seg_ques))
val_size = int(0.1 * len(seg_ques))
test_size = len(seg_ques) - train_size - val_size

bert_train_data = {"ques": seg_ques[:train_size], "ans": seg_ans[:train_size]}
bert_val_data = {"ques": seg_ques[train_size:train_size+val_size], "ans": seg_ans[train_size:train_size+val_size]}
bert_test_data = (seg_ques[train_size+val_size:],seg_ans[train_size+val_size:])

# Convert the data to a Hugging Face Dataset object
bert_train_dataset = Dataset.from_dict(bert_train_data)
bert_val_dataset = Dataset.from_dict(bert_val_data)

In [14]:
from transformers import BertTokenizer
# Initialize the BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the input and output sequences
def tokenize_function(examples):
    # Tokenize the English text
    source_texts = examples['ques']
    tokenized_inputs = bert_tokenizer(
        source_texts,
        padding=False,
        truncation=True,
        max_length=512
    )

    # Tokenize the Vietnamese text
    target_texts = examples['ans']
    tokenized_targets = bert_tokenizer(
        target_texts,
        padding=False,
        truncation=True,
        max_length=512
    )

    # Add the tokenized Vietnamese text as targets to the tokenized English text as inputs
    tokenized_inputs['labels'] = tokenized_targets['input_ids']

    return tokenized_inputs

bert_train_dataset = bert_train_dataset.map(tokenize_function, batched=True)
bert_val_dataset = bert_val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/87944 [00:00<?, ? examples/s]

Map:   0%|          | 0/10993 [00:00<?, ? examples/s]

In [15]:
from transformers import BertConfig, EncoderDecoderConfig, Seq2SeqTrainingArguments , EncoderDecoderModel

# Initialize the Encoder-Decoder configuration
config = EncoderDecoderConfig.from_encoder_decoder_configs(
    encoder_config=BertConfig.from_pretrained('bert-base-multilingual-cased'),
    decoder_config=BertConfig.from_pretrained('bert-base-multilingual-cased')
)
config.decoder_start_token_id = bert_tokenizer.cls_token_id
config.pad_token_id = bert_tokenizer.pad_token_id

# Initialize the Encoder-Decoder model
bert = EncoderDecoderModel.from_pretrained('/content/drive/MyDrive/NLP/checkpoint/bert/checkpoint-27485', config=config)   #/content/drive/MyDrive/NLP/checkpoint/bert/checkpoint-27485  , bert-base-multilingual-cased
checkpoint = '/content/drive/MyDrive/NLP/checkpoint/bert/' 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
# Define the training arguments
load_trained = True

if load_trained == False:
  training_args = Seq2SeqTrainingArguments(
      predict_with_generate=True,
      evaluation_strategy='epoch',
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=5, # Modify epoch 
      learning_rate=5e-5,
      save_total_limit=2,
      fp16= True if device.type == 'cuda' else False,  # True if use CUDA , False if use CPU
      push_to_hub=False,
      logging_steps=1000,
      save_steps=27485,
      output_dir= checkpoint,
      overwrite_output_dir=True
  )

  # Initialize the Seq2SeqTrainer and start training
  data_collator = DataCollatorForSeq2Seq(tokenizer=bert_tokenizer, model=bert)

  trainer = Seq2SeqTrainer(
      model=bert,
      args=training_args,
      train_dataset=bert_train_dataset,
      data_collator=data_collator,
      eval_dataset=bert_val_dataset
  )

  trainer.train()
else:
  # Initialize the BERT tokenizer
  bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
  # Initialize the Encoder-Decoder model
  bert = EncoderDecoderModel.from_pretrained('/content/drive/MyDrive/NLP/checkpoint/bert/checkpoint-27485', config=config).to(device)

In [21]:
from torchtext.data.metrics import bleu_score
# Đánh giá mô hình bằng độ đo BLEU
evaluate = True
preds = []
refs = []
total = len(bert_test_data)
i = 0
######
if evaluate:
  for ques, ans in zip(seg_ques[train_size+val_size:],seg_ans[train_size+val_size:]):
    print("Evaluation state : %0.3f %% " % (i/total))

    input_encodings = bert_tokenizer([ques], truncation=True,padding = True, max_length=512, return_tensors='pt')

    # Generate predicted answer
    output_encodings = bert.generate(input_ids=input_encodings['input_ids'].to(device), 
                                      attention_mask=input_encodings['attention_mask'].to(device),
                                      num_beams=1, 
                                      max_length=512,)

    # Decode the predicted answer
    output_sequences = [bert_tokenizer.decode(output_encoding, skip_special_tokens=True) for output_encoding in output_encodings]
    
    preds.extend(output_sequences)
    refs.extend([ans])

    i += 1
    clear_output(wait=True)

  print('Done.!')

Evaluation state : 1300.000 % 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 9>:16                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1515 in generate        │
│                                                                                                  │
│   1512 │   │   │   │   )                                                                         │
│   1513 │   │   │                                                                                 │
│   1514 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1515 │   │   │   return self.greedy_search(                                                    │
│   1516 │   │   │   │   input_ids,                                                                │
│   1517 │   │   │   │   logits_processor=logits_processor,                                        │
│   1518 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2332 in greedy_search   │
│                                                                                                  │
│   2329 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2330 │   │   │                                                                                 │
│   2331 │   │   │   # forward pass to get next token                                              │
│ ❱ 2332 │   │   │   outputs = self(                                                               │
│   2333 │   │   │   │   **model_inputs,                                                           │
│   2334 │   │   │   │   return_dict=True,                                                         │
│   2335 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used    

In [22]:
# Kiểm tra câu dự đoán
if evaluate:
  print(refs[:10])
  print(preds[:10])

['người Do Thái', 'On the Night of the Fire ( 1939 ) ,', 'tháng 3 năm 2011', 'On the Freedom of a Christian .', 'định_hướng công_nghệ mạnh hơn', 'Daewoo', 'đế_chế thuộc_địa', 'Tesla và / hoặc Edison đã từ_chối giải_thưởng', '1970', '18 - karat mạ vàng lớn']
['một _ số số _ nhân có liên _ quan đến việc cung _ cấp các công _ cụ tài _ chính _ lại _ lại, nhưng không có lợi _ ích cho một công _ ty thể xuất _ hiện có _ lại _ thành _ lại. _ lại. có chung là một lúc _ đi. có chung là một lúc với việc bắt _ lại họ đi.. _ lại _ lại họ có liên _ đó có về mặt _ trường về việc đi. có nhiều người có chung. _ lập _ đó là một lúc. _ đó, nhưng nó có liên _ đó là người khác. có chung là một lúc lại họ có với việc _ đó là người khác. _ đó _ là người khác _ đó là người khác _ là người khác có _ đó là người khác và bản _ là một lúc.. có chung. có _ là người. có chung _ đó là người khác _ là người khác. dị tính ra lại họ có chung _ là người không? có liên _ là người. và bản đó là người khác. có liên _ đó _ 

In [23]:
# Compute the BLEU score
if evaluate:
  chencherry = SmoothingFunction()
  score = corpus_bleu(refs,preds, smoothing_function=chencherry.method2)
  print(f"BLEU score: {score*100:.4f}")

BLEU score: 0.0112


# ***Building bot***

In [33]:
def process_data(data,data_type):
  #Chuẩn hóa dữ liệu tiếng việt
  norm_data = text_normalize(data)

  # Sử dụng word segmentation
  seg_data = word_tokenize(norm_data, format="text")

  # Biểu diễn các câu bằng vị trí có trong từ điển
  data_index = []
  for word in seg_data.split():
    if word in word2index:
      data_index.append(word2index[word])

  # Padding câu về max length
  data_pad = data_index[:max_length] + [word2index['<pad>']] * (max_length - len(data_index[:max_length]))
  
  if data_type == 0:
    return torch.LongTensor(data_pad).view(-1,1)
  elif data_type == 1:
    return data_pad
  elif data_type == 2:
    return seg_data
    
  return None

In [44]:
def ChatBot(user_input,model_name):
  predict = ""
  
  if model_name == 'seq2seq':
    tensor_ques = process_data(user_input,0).to(device)
    output = seq2seq(tensor_ques,tensor_ques, teacher=0) 
    output = output.permute(1,0,2).argmax(dim=-1).cpu().numpy()
    predict = " ".join(decode_word(index2word,output)[0])

  if model_name == 'seq2seqAttention':
    tensor_ques = process_data(user_input,0).to(device)
    output = seq2seqAttention(tensor_ques,tensor_ques, teacher=0) 
    output = output.permute(1,0,2).argmax(dim=-1).cpu().numpy()
    predict = " ".join(decode_word(index2word,output)[0])

  if model_name == 'bert':
    input_encodings = bert_tokenizer([process_data(user_input,2)], truncation=True,padding = True, max_length=512, return_tensors='pt')

    # Generate predicted answer
    output_encodings = bert.generate(input_ids=input_encodings['input_ids'].to(device), 
                                      attention_mask=input_encodings['attention_mask'].to(device),
                                      num_beams=4, 
                                      max_length=512,)

    # Decode the predicted answer
    output_sequences = [bert_tokenizer.decode(output_encoding, skip_special_tokens=True) for output_encoding in output_encodings]
    predict = output_sequences[0]

  return predict

Testing with 2 sentence

In [42]:
short_sen = 'Bạn là ai?'
print('seq2seq > ',ChatBot(short_sen,'seq2seq'))
print('seq2seqAttention > ',ChatBot(short_sen,'seq2seqAttention'))
print('bert > ',ChatBot(short_sen,'bert'))

seq2seq >  mongol
seq2seqAttention >  quý_tộc wrotham ,
bert >  một _ số số _ nhân có liên _ quan đến việc cung _ cấp các công _ cụ tài _ chính _ lại _ lại, nhưng không có lợi _ ích cho một công _ ty thể xuất _ hiện có _ lại _ thành _ lại. _ lại. có chung là một lúc _ đi. có chung là một lúc với việc bắt _ lại họ đi.. _ lại _ lại họ có liên _ đó có về mặt _ trường về việc đi. có nhiều người có chung. _ lập _ đó là một lúc. _ đó, nhưng nó có liên _ đó là người khác. có chung là một lúc lại họ có với việc _ đó là người khác. _ đó _ là người khác _ đó là người khác _ là người khác có _ đó là người khác và bản _ là một lúc.. có chung. có _ là người. có chung _ đó là người khác _ là người khác. dị tính ra lại họ có chung _ là người không? có liên _ là người. và bản đó là người khác. có liên _ đó _ là người khác. có liên _ là người dị tính ra lại tình _ sao có đối _ thị _ là người. và bản đó. dị tính này là người. dõi đó là người khác - và bản _ đó là một lúc này, nhưng nó đã nhớ lại tình _ 

In [43]:
long_sen = 'Bạn có thể cho tôi biết vấn đề về thế giới hiện này được không'
print('seq2seq > ',ChatBot(long_sen,'seq2seq'))
print('seq2seqAttention > ',ChatBot(long_sen,'seq2seqAttention'))
print('bert > ',ChatBot(long_sen,'bert'))

seq2seq >  được
seq2seqAttention >  ok bạn nhá nhé .
bert >  một _ số số _ nhân có liên _ quan đến việc cung _ cấp các công _ cụ tài _ chính _ lại _ lại, nhưng không có lợi _ ích cho một công _ ty thể xuất _ hiện có _ lại _ thành _ lại. _ lại. có chung là một lúc _ đi. có chung là một lúc với việc bắt _ lại họ đi.. _ lại _ lại họ có liên _ đó có về mặt _ trường về việc đi. có nhiều người có chung. _ lập _ đó là một lúc. _ đó, nhưng nó có liên _ đó là người khác. có chung là một lúc lại họ có với việc _ đó là người khác. _ đó _ là người khác _ đó là người khác _ là người khác có _ đó là người khác và bản _ là một lúc.. có chung. có _ là người. có chung _ đó là người khác _ là người khác. dị tính ra lại họ có chung _ là người không? có liên _ là người. và bản đó là người khác. có liên _ đó _ là người khác. có liên _ là người dị tính ra lại tình _ sao có đối _ thị _ là người. và bản đó. dị tính này là người. dõi đó là người khác - và bản _ đó là một lúc này, nhưng nó đã nhớ lại tình _ đó.

Testing ChatBot

In [38]:
model_name = 'seq2seqAttention'
while(True):
  user_input = input('You : ')
  if user_input == 'q':
    break
  bot = ChatBot(user_input,model_name)
  print('ChatBot :',bot)

clear_output()